<a href="https://colab.research.google.com/github/halfag/LocalLLM/blob/main/Mixtral_DnD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!apt-get update -q && apt install pciutils -y

!curl https://ollama.ai/install.sh | sh
!nohup ollama serve &




In [2]:
#%%capture
!ollama pull mixtral:8x7b

Streaming output truncated to the last 5000 lines.
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling 79b7eca19f7a... 100% ▕▏   43 B                         
pulling ed11eda7790d... 100% ▕▏   30 B                         
pulling 9dec05e9b2db... 100% ▕▏  484 B                         
verifying sha256 digest ⠸ pulling manifest 
pulling e9e56e8bb5f0... 100% ▕▏  26 GB                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling 79b7eca19f7a... 100% ▕▏   43 B                         
pulling ed11eda7790d... 100% ▕▏   30 B                         
pulling 9dec05e9b2db... 100% ▕▏  484 B                         
verifying sha256 digest ⠼ pulling manifest 
pulling e9e56e8bb5f0... 100% ▕▏  26 GB                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling 79b7eca19f7a... 100% ▕▏   43 B                         
pulling ed11eda7790d... 100% ▕▏   30 B                         
pulling 9dec05e9b2db... 100% 

In [ ]:
%%capture
!pip install kaleido python-multipart cohere
!pip install litellm[proxy_server] pyautogen


In [3]:
%%capture
!nohup litellm --model ollama/mixtral:8x7b &

In [ ]:
!pip install aiohttp pyngrok
!ngrok config add-authtoken 2YyeNPLWs82ZVtL13BIV3rZldWj_68suxzPW6XuS7F1fGfeko

import os
import asyncio

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )

await asyncio.gather(
    run_process(['ollama', 'serve']),
    run_process(['ngrok', 'http', '--log', 'stderr', '11434']),
)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
>>> starting ollama serve
>>> starting ngrok http --log stderr 11434
Error: listen tcp 127.0.0.1:11434: bind: address already in use
t=2023-12-21T16:42:40+0000 lvl=info msg="no configuration paths supplied"
t=2023-12-21T16:42:40+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
t=2023-12-21T16:42:40+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
t=2023-12-21T16:42:40+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
t=2023-12-21T16:42:41+0000 lvl=info msg="client session established" obj=tunnels.session obj=csess id=538f0b1481cb
t=2023-12-21T16:42:41+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2023-12-21T16:42:41+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=http://localhost:11434 url=https://9bb0-34-142-205-14.ngrok-free.app


In [ ]:
# Code used:
import autogen
import openai
openai.api_type = "open_ai"
openai.api_key = "sk-111111111111111111111111111111111111111111111"
openai.api_base = "http://127.0.0.1:8000/v1"
# openai.api_version = "2023-05-15"


wizard_config_list = [
    {
        "api_key": "sk-111111111111111111111111111111111111111111111",
        #"api_key": "NULL",
        # 'api_type': 'open_ai',
        'base_url': "http://127.0.0.1:8000/v1",
        # 'api_version': '2023-05-15',
        # "timeout": "600",
        # "request_timeout": 8000
    },
]

llama2_config_list = [
    {
        "base_url": "http://127.0.0.1:8000/v1",
        #"api_key": "NULL",
        "api_key": "sk-111111111111111111111111111111111111111111111",
        # 'api_type': 'open_ai',
        # "timeout": "600",
        # "request_timeout": 8000
    },
]


coder_llm_config = {
    # "seed": None,  # change the seed for different trials
    "temperature": 0,
    "config_list": llama2_config_list,
    # "timeout": 600,
    # "request_timeout": 8000
    # "clear_cache": True
}

llama2_llm_config = {
    # "seed": None,  # change the seed for different trials
    # no caching
    # "clear_cache": True,
    "temperature": 0,
    "config_list": llama2_config_list,
    # "timeout": 600,
    # "request_timeout": 8000
}


user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message = """Reply TERMINATE if the task has been solved at full satisfaction. Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat", "use_docker":"True"},
   human_input_mode="TERMINATE"
)
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llama2_llm_config
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Manage deveolpment of software",
    llm_config=llama2_llm_config
)
groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llama2_llm_config)



In [ ]:
user_proxy.initiate_chat(manager, message="""What's today's date?""")
# type exit to terminate the chat

Trying CrewAI












In [4]:
!pip install crewai fastapi kaleido python-multipart uvicorn cohere tiktoken typing-extensions sec_api unstructured openai faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.3/794.3 kB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import os

import requests

from langchain.tools import tool
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

from sec_api import QueryApi
from unstructured.partition.html import partition_html


from langchain.llms import Ollama
ollama_openhermes = Ollama(model="mixtral:8x7b")


class SECTools():
  @tool("Search 10-Q form")
  def search_10q(data):
    """
    Useful to search information from the latest 10-Q form for a
    given stock.
    The input to this tool should be a pipe (|) separated text of
    length two, representing the stock ticker you are interested, what
    question you have from it.
		For example, `AAPL|what was last quarter's revenue`.
    """
    stock, ask = data.split("|")
    queryApi = QueryApi(api_key="cecb4fc03aacfe47839670320ea3dab9ec108fdcdb3945666f722b8dd05c4106")
    query = {
      "query": {
        "query_string": {
          "query": f"ticker:{stock} AND formType:\"10-Q\""
        }
      },
      "from": "0",
      "size": "1",
      "sort": [{ "filedAt": { "order": "desc" }}]
    }

    filings = queryApi.get_filings(query)['filings']
    link = filings[0]['linkToFilingDetails']
    answer = SECTools.__embedding_search(link, ask)
    return answer

  @tool("Search 10-K form")
  def search_10k(data):
    """
    Useful to search information from the latest 10-K form for a
    given stock.
    The input to this tool should be a pipe (|) separated text of
    length two, representing the stock ticker you are interested, what
    question you have from it.
    For example, `AAPL|what was last year's revenue`.
    """
    stock, ask = data.split("|")
    queryApi = QueryApi(api_key="cecb4fc03aacfe47839670320ea3dab9ec108fdcdb3945666f722b8dd05c4106")
    query = {
      "query": {
        "query_string": {
          "query": f"ticker:{stock} AND formType:\"10-K\""
        }
      },
      "from": "0",
      "size": "1",
      "sort": [{ "filedAt": { "order": "desc" }}]
    }

    filings = queryApi.get_filings(query)['filings']
    link = filings[0]['linkToFilingDetails']
    answer = SECTools.__embedding_search(link, ask)
    return answer

  def __embedding_search(url, ask):
    text = SECTools.__download_form_html(url)
    elements = partition_html(text=text)
    content = "\n".join([str(el) for el in elements])
    text_splitter = CharacterTextSplitter(
        separator = "\n",
        chunk_size = 1000,
        chunk_overlap  = 150,
        length_function = len,
        is_separator_regex = False,
    )
    docs = text_splitter.create_documents([content])
    retriever = FAISS.from_documents(
      docs, OpenAIEmbeddings()
    ).as_retriever()
    answers = retriever.get_relevant_documents(ask, top_k=4)
    answers = "\n\n".join([a.page_content for a in answers])
    return answers

  def __download_form_html(url):
    headers = {
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
      'Accept-Encoding': 'gzip, deflate, br',
      'Accept-Language': 'en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7',
      'Cache-Control': 'max-age=0',
      'Dnt': '1',
      'Sec-Ch-Ua': '"Not_A Brand";v="8", "Chromium";v="120"',
      'Sec-Ch-Ua-Mobile': '?0',
      'Sec-Ch-Ua-Platform': '"macOS"',
      'Sec-Fetch-Dest': 'document',
      'Sec-Fetch-Mode': 'navigate',
      'Sec-Fetch-Site': 'none',
      'Sec-Fetch-User': '?1',
      'Upgrade-Insecure-Requests': '1',
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    return response.text

In [8]:
import json
import os

import requests
from langchain.tools import tool

# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

class SearchTools():
  @tool("Search the internet")
  def search_internet(query):
    """Useful to search the internet
    about a a given topic and return relevant results"""
    top_result_to_return = 4
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': "9de5f1f784f494e8058f3c350e720462127e7c92",
        'content-type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    results = response.json()['organic']
    string = []
    for result in results[:top_result_to_return]:
      try:
        string.append('\n'.join([
            f"Title: {result['title']}", f"Link: {result['link']}",
            f"Snippet: {result['snippet']}", "\n-----------------"
        ]))
      except KeyError:
        next

    return '\n'.join(string)

  @tool("Search news on the internet")
  def search_news(query):
    """Useful to search news about a company, stock or any other
    topic and return relevant results"""""
    top_result_to_return = 4
    url = "https://google.serper.dev/news"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': "9de5f1f784f494e8058f3c350e720462127e7c92",
        'content-type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    results = response.json()['news']
    string = []
    for result in results[:top_result_to_return]:
      try:
        string.append('\n'.join([
            f"Title: {result['title']}", f"Link: {result['link']}",
            f"Snippet: {result['snippet']}", "\n-----------------"
        ]))
      except KeyError:
        next

    return '\n'.join(string)


In [9]:
from langchain.tools import tool


class CalculatorTools():

  @tool("Make a calcualtion")
  def calculate(operation):
    """Useful to perform any mathematica calculations,
    like sum, minus, mutiplcation, division, etc.
    The input to this tool should be a mathematical
    expression, a couple examples are `200*7` or `5000/2*10`
    """
    return eval(operation)

In [10]:
import json
import os

import requests
from crewai import Agent, Task
from langchain.tools import tool
from unstructured.partition.html import partition_html

os.environ["OPENAI_API_KEY"] = "sk-111111111111111111111"
from langchain.chat_models import ChatOpenAI
from langchain.llms import Ollama
ollama_openhermes = Ollama(model="mixtral:8x7b")

class BrowserTools():

  @tool("Scrape website content")
  def scrape_and_summarize_website(website):
    """Useful to scrape and summarize a website content"""
    url = f"https://chrome.browserless.io/content?token='3d11f297-9b55-4298-97f2-255f6ed89ab8'"
    payload = json.dumps({"url": website})
    headers = {'cache-control': 'no-cache', 'content-type': 'application/json'}
    response = requests.request("POST", url, headers=headers, data=payload)
    elements = partition_html(text=response.text)
    content = "\n\n".join([str(el) for el in elements])
    content = [content[i:i + 8000] for i in range(0, len(content), 8000)]
    summaries = []
    for chunk in content:
      agent = Agent(
          role='Principal Researcher',
          llm=ollama_openhermes,
          goal=
          'Do amazing research and summaries based on the content you are working with',
          backstory=
          "You're a Principal Researcher at a big company and you need to do research about a given topic.",
          allow_delegation=False),
      task = Task(
          agent=agent,
          description=
          f'Analyze and summarize the content below, make sure to include the most relevant information in the summary, return only the summary nothing else.\n\nCONTENT\n----------\n{chunk}'
      )
      summary = task.execute()
      summaries.append(summary)
    return "\n\n".join(summaries)

In [11]:
from google.colab import userdata


# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = "sk-111111111111111111111"
# from langchain.chat_models import ChatOpenAI
from langchain.llms import Ollama
ollama_openhermes = Ollama(model="mixtral:8x7b")
# ollama_openhermes = llm=ChatOpenAI(temperature=0.7, model_name="gpt-4")
# Pass Ollama Model to Agents: When creating your agents within the CrewAI framework, you can pass the Ollama model as an argument to the Agent constructor. For instance:


from crewai import Agent, Task, Crew, Process

# Define your agents with roles and goals
researcher = Agent(
  role='Researcher',
  goal='Discover new insights',
  backstory="You're a world class researcher working on a major data science company",
  verbose=True,
  allow_delegation=False,
  llm=ollama_openhermes,
  tools=[
    BrowserTools.scrape_and_summarize_website,
    SearchTools.search_internet,
    # CalculatorTools.calculate,
    # SECTools.search_10q,
    # SECTools.search_10k
  ]
)
writer = Agent(
  role='Writer',
  goal='Create engaging content',
  backstory="You're a famous technical writer, specialized on writing data related content",
  verbose=True,
  allow_delegation=False,
  llm=ollama_openhermes
)

# Create tasks for your agents
task1 = Task(description='Investigate the cyber attack group "volt typhoon"', agent=researcher)
task2 = Task(description='write a very long article about "volt typhoon", which includes both strategic and tactical informaion', agent=writer)

# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=2, # Crew verbose more will let you know what tasks are being worked on, you can set it to 1 or 2 to different logging levels
  process=Process.sequential # Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
)

# Get your crew to work!
result = crew.kickoff()

Working Agent: Researcher
Starting Task: Investigate the cyber attack group "volt typhoon" ...


> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? Yes
Action: Search the internet
Action Input: "volt typhoon cyber attack group"Title: What is Volt Typhoon, the alleged China-backed hacking group? - Reuters
Link: https://www.reuters.com/technology/what-is-volt-typhoon-alleged-china-backed-hacking-group-2023-05-25/
Snippet: Reporter covering cybersecurity, surveillance, and disinformation for Reuters. Work has included investigations into state-sponsored espionage, ...

-----------------
Title: China's cyber intrusions have hit ports and utilities, officials say - The Washington Post
Link: https://www.washingtonpost.com/technology/2023/12/11/china-hacking-hawaii-pacific-taiwan-conflict/
Snippet: Morgan Adamski, director of the National Security Agency's Cybersecurity Collaboration Center, confirmed in an email that Volt Typhoon activity ...

-----------------
Title: V

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


ValidationError: ignored